In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    # 保留数字
    review_text = re.sub("[^a-zA-Z0-9]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 去掉empty word
    words = filter(lambda x: x, words)
    # 5. Return a list of words
    return(words)

In [30]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
# 英文的标点符号
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    try:
        raw_sentences = tokenizer.tokenize(review.strip().decode('utf-8'))
    except:
        print review
        raise 1;
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            words = review_to_wordlist( raw_sentence, \
              remove_stopwords )
            if words:
                sentences.append(words)
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [31]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review)

print "Parsing sentences from test set"
for review in test["review"]:
    sentences += review_to_sentences(review)
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review)

Parsing sentences from training set


/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


Parsing sentences from test set


/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % m

In [33]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...


In [34]:
model.most_similar("man")

[(u'woman', 0.6247764229774475),
 (u'lad', 0.5895715951919556),
 (u'lady', 0.5756250619888306),
 (u'chap', 0.568739652633667),
 (u'monk', 0.5380529165267944),
 (u'businessman', 0.5250462889671326),
 (u'guy', 0.5208761692047119),
 (u'millionaire', 0.5175275802612305),
 (u'farmer', 0.5151277780532837),
 (u'men', 0.5131310224533081)]

In [35]:
model.most_similar("queen")

[(u'princess', 0.6513222455978394),
 (u'goddess', 0.6247842907905579),
 (u'bride', 0.5992112159729004),
 (u'belle', 0.586050271987915),
 (u'maid', 0.5686043500900269),
 (u'dame', 0.5681019425392151),
 (u'countess', 0.5583339929580688),
 (u'regina', 0.5412328243255615),
 (u'duchess', 0.5403283834457397),
 (u'nun', 0.532714307308197)]

In [36]:
model.most_similar("awful")

[(u'terrible', 0.7806628942489624),
 (u'atrocious', 0.7442105412483215),
 (u'horrible', 0.7295991778373718),
 (u'dreadful', 0.7285383343696594),
 (u'abysmal', 0.7068103551864624),
 (u'appalling', 0.6946099996566772),
 (u'horrid', 0.6860538125038147),
 (u'horrendous', 0.6783500909805298),
 (u'lousy', 0.6520316004753113),
 (u'bad', 0.6105291843414307)]

In [45]:
len(model.index2word)

19545